# Home Assignment 01. OpenAI, ElasticSearch, RAG

### Q1. running Elastic

**Answer**: `42f05b9372a9a4a470db3b52817899b99a76ee73`

In [2]:
!curl elasticsearch:9200

{
  "name" : "33e592aa075a",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "fUjwSc3BT9KhhbpJky3U9w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [7]:
import json
from pathlib import Path
import sys

PROJECT_DIR = Path(".").absolute().parent.parent
sys.path.append(str(PROJECT_DIR))

#### Getting the Data

In [8]:
DATA_DIR = Path(PROJECT_DIR, "data")
assert DATA_DIR.exists()

# read all documentds:
with open(Path(DATA_DIR, "documents.json"), "rt") as f_in:
    docs_raw = json.load(f_in)

# collect all documents from all the courses
documents = []
for course_dict in docs_raw:
    course_name = course_dict["course"]
    for doc in course_dict["documents"]:
        doc["course"] = course_name
        documents.append(doc)

### Q2. Indexing the Data

**Answer**: `index`

In [11]:
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [10]:
es_client = Elasticsearch("http://elasticsearch:9200")

In [12]:
# Elastic config definition
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-descripiton"

# create index
es_client.indices.create(
    index=index_name,
    body=index_settings
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-descripiton'})

In [13]:
# Add document to index one by one:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 508.20it/s]


### Q3: Searching

**Answer**: `84.050095`

In [18]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    # "fields": ["question^4", "text", "section"],
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
            # "filter": {
            #     "term": {
            #         "course": "data-engineering-zoomcamp"
            #     }
            # },
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response["hits"]

{'total': {'value': 865, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-descripiton',
   '_id': 'HFP2PpAB-gLAdVk5eC-t',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}}]}

### Q4: Filtering

**Answer**: `'How do I copy files from a different folder into docker container’s working directory?'`

In [19]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    # "fields": ["question^4", "text", "section"],
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            },
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response["hits"]

{'total': {'value': 345, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-descripiton',
   '_id': 'HFP2PpAB-gLAdVk5eC-t',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-descripiton',
   '_id': 'O1P2PpAB-gLAdVk5eC_g',
   '_score': 51.04628,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can 

### Q5: Building a prompt

**Answer**: `1462`

In [25]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [27]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [26]:
context_qna = []
for hit in response["hits"]["hits"]:
    source_doc = hit["_source"]
    context_i = context_template.format(
        question=source_doc["question"],
        text=source_doc["text"],
    )
    context_qna.append(context_i)

In [28]:
context = "\n\n".join(context_qna)

question = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=question, context=context)
len(prompt)

1462

### Q6: Tokens

**Answer**: `322`

In [29]:
import tiktoken

In [31]:
# tiktoken - tokenization used by OpenAI
encoding = tiktoken.encoding_for_model(model_name="gpt-4o")

# how many tokens does our prompt have?
prompt_tokens = encoding.encode(prompt)
len(prompt_tokens)

322

In [35]:
# to decode back a single word:
encoding.decode_single_token_bytes(150339)

b' QUESTION'

### Bonus: Generating the answer

In [42]:
from openai import OpenAI

from project.settings.openai import openai_settings

In [43]:
# create OpenAI client:
openai_client = OpenAI(api_key=openai_settings.api_key.get_secret_value())

In [48]:
def elastic_search(
    query: str,
    size: int = 5,
    question_weight: int = 3,
    course: str = "machine-learning-zoomcamp"
) -> list[dict]:
    
    # setup elastic request
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        # "fields": [f"question^{question_weight}", "text", "section"],
                        "fields": [f"question^{question_weight}", "text"],
                        "type": "best_fields",
                    }
                },
                "filter": {
                    "term": {
                        "course": f"{course}"
                    }
                },
            }
        }
    }
    
    # search in elastic
    response = es_client.search(index=index_name, body=search_query)
    
    # save initial documents found as doc list:
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [49]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


def build_prompt(query: str, search_results: [dict]) -> str:
    context_qna = []
    for source_doc in search_results:
        context_i = context_template.format(
            question=source_doc["question"],
            text=source_doc["text"],
        )
        context_qna.append(context_i)

    context = "\n\n".join(context_qna)
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [50]:
def llm(prompt: str) -> str:
    """Request to ChatGPT API to make a query"""
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )
    return response.choices[0].message.content

In [51]:
def rag(query: str) -> str:
    # 1. search in elastic
    faq_results = elastic_search(query, question_weight=4, size=3)
    # 2. build a prompt
    prompt = build_prompt(query, search_results=faq_results)
    # 3. ask OpenAI
    answer = llm(prompt)
    return answer

In [53]:
query = "How do I execute a command in a running docker container?"
answer = rag(query=query)

In [57]:
# answer n tokens:
len(encoding.encode(answer))

134

In [54]:
print(answer)

To execute a command in a running docker container, you can use the `docker exec` command. First, identify the container's ID by using the `docker ps` command, then execute the desired command within that container using the `docker exec` command. Here are the steps:

1. List all running containers to find the container ID:
   ```sh
   docker ps
   ```

2. Execute a command inside the running container (for example, starting a bash shell):
   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of the container you want to interact with.


### Bonus: calculating the costs

In [64]:
average_question_tokens = 150
average_answer_tokens = 250
n_requests = 1_000

# prices gpt-4o, gpt 3.5 is x10 cheaper
# Details: https://openai.com/api/pricing/

# Input: US$5.00 / 1M tokens
price_per_question_token = 5 / 1e6
# Output: US$15.00 / 1M tokens
price_per_answer_token = 15 / 1e6

total_cost = n_requests * (
    average_question_tokens * price_per_question_token
    + average_answer_tokens * price_per_answer_token
)
print(f"Total costs: ${total_cost:.2f}")

Total costs: $4.50
